<h1>Clean, Dirty Plate Splitting System With Deep Learning</h1>
This project aims to solve plastic recycling porblem. Perfect recycling by human isn't done, thus majority of the plastics are being disposed of due to contamination.<br>
With synthetic data generation and deep learning, we aim to create a system that can collect contaminated plastics and increase recycling rate.

In [ ]:
# jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from pxr import PhysicsSchemaTools, Gf, Usd, UsdGeom, UsdPhysics

import omni
from omni.isaac.core.utils.stage import add_reference_to_stage
from omni.isaac.core.utils.prims import get_prim_at_path, is_prim_path_valid
from omni.isaac.core import World
from omni.physx.scripts import utils as physx_utils

import omni.kit.commands as cmds
import omni.kit.viewport.utility as vp_utils
import omni.kit.app

#from IPython.display import display

import numpy as np
import json
import time
import random

<h2><font color='DimGray'>Load Stage</font></h2>

In [ ]:
root_dir = "/home/kijung914/myisaac/myfiles"

In [ ]:
# Opening a Stage
usd_context = omni.usd.get_context()
omni.kit.window.file.open_stage(f"{root_dir}/usd_files/stage.usd") #Load Stage

# WAIT for world to load!!

In [ ]:
app = omni.kit.app.get_app()
stage = omni.usd.get_context().get_stage()

world = World(stage_units_in_meters=1.0) #Set World
#world.reset()
random.seed(42) #Set Random Seed

<h2><font color='red'>Timer</font></h2>
This makes interrupts(timers) for the omniverse simulation and executes a callback function.

In [ ]:
class tmr:
    def __init__(self,func, dt, name, num = None, func_on_exit = None):
        self.name = name # timer name
        self.dt = dt # interval
        self.counter = 0 # counter
        self.func = func # do what on callback
        
        if func_on_exit is None:
            self.func_on_exit = lambda:None
        else:
            self.func_on_exit = func_on_exit # do something on right before end of callback
            
        self.num = num # max amount of execution
        self.cnt = 0 # current amount of execution
        
        self.sub = None # Omniverse Event Subscriber

        self.running = False

    def begin(self):
        if not self.running: # If there's no subcriber
            try:
                update_stream = omni.kit.app.get_app().get_update_event_stream() # Subscription to frame event
                self.sub = update_stream.create_subscription_to_pop(self.callback, name=self.name)
                carb.log_warn(f"{self.name} subscription initiated")
                self.running = True
            except Exception as err:
                carb.log_warn(err)
                
    def stop(self): # Kill subscriber
        try:
            self.running = False
            self.sub.unsubscribe()
            self.sub = None
            carb.log_warn(f"Unsubcribed {self.name}")
            self.func_on_exit()
        except Exception as err:
            carb.log_warn(err)
            
    def run(self):
        self.func()
    
    def callback(self, e: carb.events.IEvent): # Callback
        dt = e.payload["dt"]
        self.counter += dt #obtain time from omniverse

        if self.counter > self.dt: # If sufficient time has passed
            self.counter = 0

            if self.num is not None: # If there's max amount of iteration
                if self.cnt >= self.num:
                    self.stop() # End Timer
                    return
                else:
                    self.cnt += 1
                    
            self.run() # Execute

    def reset(self):
        self.counter = 0

    def __del__(self):
        self.stop()

<h2><font color='DimGray'>Conveyor Class</font></h2>
This Controls Speed of available conveyor prims.

In [ ]:
class conveyors:
    def __init__(self):
        self.convs = dict()
        self.read()

        self.dirs = dict()
        self.set_gain()

                
    def read(self):# Save conveyor prims to dict()
        for i in range(1,10):
            key = f"0{i}"
            self.convs[key] = get_prim_at_path(f"/World/ConveyorTrack_{key}/ConveyorBeltGraph")
        for i in range(1,3):
            key = f"1{i}"
            self.convs[key] = get_prim_at_path(f"/World/ConveyorTrack_{key}/ConveyorBeltGraph")

        self.splitter = [get_prim_at_path("/World/ConveyorTrack_01/ConveyorBeltGraph"), get_prim_at_path("/World/ConveyorTrack_01/ConveyorBeltGraph_01")]

    def set_gain(self, gain=1.0): # Set Speed Gain for all conveyors
        self.gain = gain
        #first_camera_capturer
        self.dirs['01'] = -2.2 * gain
        self.dirs['02'] = -0.8 * gain
        self.dirs['03'] = 0.6 * gain
        self.dirs['04'] = 1.2 * gain
        self.dirs['05'] = 1.5 * gain
        self.dirs['06'] = 1.5 * gain
        self.dirs['07'] = 1.0 * gain
        self.dirs['08'] = 1.0 * gain
        self.dirs['09'] = 1.0 * gain
        self.dirs['11'] = -3.0
        self.dirs['12'] = 3.0

        self.run()


    def run(self, keys=None): # Move belts
        if keys is None:
            for key, prim in self.convs.items():
                prim.GetAttribute('graph:variable:Velocity').Set(self.dirs[key])

            self.splitter[1].GetAttribute('graph:variable:Velocity').Set(self.dirs['01'])
            get_prim_at_path("/World/ConveyorTrack_12/ConveyorBeltGraph_01").GetAttribute('graph:variable:Velocity').Set(3.0)

            self.let_it_go()

        else:
            for key, prim in self.convs.items():
                if key in keys:
                    prim.GetAttribute('graph:variable:Velocity').Set(self.dirs[key])
                    
    def stop(self, keys=None): # Stop belts
        if keys is None:
            for key, prim in self.convs.items():
                prim.GetAttribute('graph:variable:Velocity').Set(self.dirs[0.0])
            get_prim_at_path("/World/ConveyorTrack_12/ConveyorBeltGraph_01").GetAttribute('graph:variable:Velocity').Set(0.0)
            self.splitter[1].GetAttribute('graph:variable:Velocity').Set(0.0)
        else:
            for key, prim in self.convs.items():
                if key in keys:
                    prim.GetAttribute('graph:variable:Velocity').Set(0.0)

    def quarantine(self): # Set conveyorbelt so that the items go in loop
        self.splitter[0].GetAttribute('graph:variable:Velocity').Set(self.dirs['01'])
    def let_it_go(self): # Set conveyor so that the items go in tunnel
        self.splitter[0].GetAttribute('graph:variable:Velocity').Set(-self.dirs['01'])

<h2><font color='Green'>Plates</font></h2>
This is for (de)spawning clean/dirty plates.

In [ ]:
# Define usd_paths
dirtyplate_file_path  = f"{root_dir}/usd_files/dirty_plate.usd"
cleanplate_file_path = f"{root_dir}/usd_files/clean_plate.usd"

class plates_spawn_kill:
    def __init__(self):
        self.plates = dict() # Alive plates, prim_path: clean/dirty
        self.cnt = 0 # ID of plates
        #cannot use len(plates) since, cnt is used for unique ID, len() might cause not-uniqueness
        
        if stage.GetPrimAtPath("/World/plates").IsValid():
            cmds.execute("DeletePrims", paths=["/World/plates"])
        UsdGeom.Scope.Define(stage, "/World/plates")
        

    def spawn_dirty_plate(self, pos = 'start'):
        usd_path = dirtyplate_file_path
        prim_path = f"/World/plates/dirty_{self.cnt}"
        
        add_reference_to_stage(usd_path=usd_path, prim_path=prim_path) # Spawn Plate
        
        xform = UsdGeom.Xformable(get_prim_at_path(prim_path)) # Apply required transforms
        scale_op = xform.AddScaleOp()
        rotate_op = xform.AddRotateXYZOp()
        translate_op = xform.AddTranslateOp()
        if pos == 'camera':
            translate_op.Set(Gf.Vec3f(-5, 0, 1.8))
        elif pos == 'start':
            translate_op.Set(Gf.Vec3f(-13, 0, 1.8))

        # Add Physics
        UsdPhysics.RigidBodyAPI.Apply(get_prim_at_path(prim_path))
        UsdPhysics.CollisionAPI.Apply(get_prim_at_path(f"{prim_path}/myroot/items/myplate"))
    
        self.plates[prim_path] = ['dp', get_prim_at_path(prim_path)] # Save as dirty
        self.cnt += 1
        
    def spawn_clean_plate(self, pos = 'start'):
        usd_path = cleanplate_file_path
        prim_path = f"/World/plates/clean_{self.cnt}"
        
        add_reference_to_stage(usd_path=usd_path, prim_path=prim_path) # Spawn Plate
        
        xform = UsdGeom.Xformable(get_prim_at_path(prim_path)) # Applt required tranforms
        scale_op = xform.AddScaleOp()
        rotate_op = xform.AddRotateXYZOp()
        translate_op = xform.AddTranslateOp()

        if pos == 'camera':
            translate_op.Set(Gf.Vec3f(-5, random.uniform(-0.21,0.16), 1.9))
        elif pos == 'start':
            translate_op.Set(Gf.Vec3f(-13, random.uniform(-0.21,0.16), 1.9))

        # Add Physics
        UsdPhysics.RigidBodyAPI.Apply(get_prim_at_path(prim_path))
        UsdPhysics.CollisionAPI.Apply(get_prim_at_path(prim_path))
    
        self.plates[prim_path] = ['p', get_prim_at_path(prim_path)] # Save as clean
        self.cnt += 1

    def force_despawn(self, prims): # despawn now!
        candidates = []
        for prim_path, [_, _] in self.plates.items():
            
            if prim_path in prims: # if current prim is in the list
                candidates.append(prim_path)
                
        cmds.execute("DeletePrims", paths=candidates)
        
        for p in candidates:
            self.plates.pop(p, None)        
    
    def should_despawn(self): # despawn as criteria
        candidates = []
        for prim_path, [key, _] in self.plates.items():
            pos = physx_utils.get_world_position(stage, prim_path)

            if pos[2] < 0.5: #if z position is below 0.5 which means it's on the ground
                candidates.append(prim_path)
                
        cmds.execute("DeletePrims", paths=candidates)
        
        for p in candidates:
            self.plates.pop(p, None)

    def all_despawn(self): # despawn all plates
        candidates = []
        for prim_path, _ in self.plates.items():
            candidates.append(prim_path)
            
        cmds.execute("DeletePrims", paths=candidates)
        
        for p in candidates:
            self.plates.pop(p, None)


<h2><font color='Green'>Bottles</font></h2>
This is for (de)spawning two types of bottles.

In [ ]:
# Define prim file paths
bottle1_path  = f"{root_dir}/usd_files/bottle1.usd"
bottle2_path  = f"{root_dir}/usd_files/bottle2.usd"

class bottle_spawn_kill:
    def __init__(self, max_objs_at_once = 30):
        self.objs = dict() # Alive bottles
        self.cnt = 0 # ID of bottles
        self.max_objs_at_once = max_objs_at_once # Limit on simultaneous alive bottles <- To reduce render loads
        
        if stage.GetPrimAtPath("/World/Bottles").IsValid():
            cmds.execute("DeletePrims", paths=["/World/Bottles"])
        UsdGeom.Scope.Define(stage, "/World/Bottles")
        
    def spawn_between_two(self): # Spawn between two types of bottles
        self.spawn(random.choice([1, 2]))
    
    def spawn(self, key, pos = 'start'): # Spawn now!
        if key == 1: # spawn bottle type 1
            usd_path = bottle1_path
        else: # spawn bottle type 2
            usd_path = bottle2_path

        prim_path = f"/World/Bottles/bottle_{self.cnt}"
        
        add_reference_to_stage(usd_path=usd_path, prim_path=prim_path) # Spawn
        
        xform = UsdGeom.Xformable(get_prim_at_path(prim_path)) # Apply necessary transforms
        scale_op = xform.AddScaleOp()
        translate_op = xform.AddTranslateOp()
        xform = UsdGeom.Xformable(get_prim_at_path(f"{prim_path}/Meshes"))
        rotate_op = xform.AddRotateXYZOp()
        
        scale_op.Set(Gf.Vec3f(0.01,0.01,0.01))
        rotate_op.Set(Gf.Vec3f(0, 0, random.randrange(-180, 180)))

        if pos == 'camera':
            translate_op.Set(Gf.Vec3f(-5200, -4.5+random.uniform(-12.0, 12.0), 187))
        elif pos == 'start':
            translate_op.Set(Gf.Vec3f(-1200, -4.5+random.uniform(-12.0, 12.0), 187))

        # Apply Physics
        UsdPhysics.RigidBodyAPI.Apply(get_prim_at_path(prim_path))
        UsdPhysics.CollisionAPI.Apply(get_prim_at_path(prim_path))
    

        self.objs[prim_path] = None # Save
        self.cnt += 1


    def should_despawn(self): # despawn bottles when the amount of bottles exceeds maximum amount
        delete_cnt = len(self.objs) - self.max_objs_at_once
        
        candidates = set()
        for i, [prim_path, _] in enumerate(self.objs.items()):
            pos = physx_utils.get_world_position(stage, prim_path)
            
            if i > delete_cnt and pos[2] > 0.5: # if there's still room and the bottle is on the conveyor, pass
                continue
                
            candidates.add(prim_path)
                
        cmds.execute("DeletePrims", paths=candidates)
        
        for p in candidates:
            self.objs.pop(p, None)

    def all_despawn(self): #despawn all bottles
        candidates = []
        for prim_path, _ in self.objs.items():
            candidates.append(prim_path)
            
        cmds.execute("DeletePrims", paths=candidates)
        
        for p in candidates:
            self.objs.pop(p, None)

<h2><font color='Purple'>Proximity Checker</font></h2>
This checks if an object is close to a point.

In [ ]:
class Proximity_Checker:
    def __init__(self, placeholder_prim_path, threshold): # Needs placeholder and a threshold for flag
        self.placeholder_prim_path = placeholder_prim_path # Placeholder (desired position)

        self.placeholder_pt = None # points of the placeholder
        self.target_pt = None # points of inquired object
        
        self.was_inside = set() # previous objs that passed threshold
        self.current_inside = set() # current objs that passed threshold
        # This can be used to check if there's been an new/left entry
        
        self.entered = set() # new entry
        self.left = set() # left entry

        self.threshold = threshold # Distance threshold (L1)

    def _get_pt(self, prim_path=None): # Obtain Point
        if prim_path is None:
            self.placeholder_pt = physx_utils.get_world_position(stage, self.placeholder_prim_path)
        else:
            self.target_pt = physx_utils.get_world_position(stage, prim_path)
            

    def _is_close(self, target_prim_path): # Check if the object is close enough (passed threshold)
        self._get_pt() # Obtain placeholder's point
        self._get_pt(target_prim_path) # Obtain the object's point

        dst = np.linalg.norm(self.placeholder_pt - self.target_pt) # Get L1 norm
        
        if dst <= self.threshold:
            return True
        else:
            return False

    def _update_set(self): # Update current to was
        self.was_inside.clear()
        self.was_inside.update(self.current_inside)
        
    def _whats_new_and_left(self): # Check new/left entry
        self.entered = self.current_inside - self.was_inside
        self.left = self.was_inside - self.current_inside

    def func_in(self, prim_path, key): # Please override
        pass

    def func_out(self): # Please override
        pass

    def func_prior(self): # Please override
        pass
    
    def run(self):
        self.current_inside = set()
        self.func_prior() # Do something before iterating
        
        for prim_path, [key, _] in pl.plates.items():
            if self._is_close(prim_path): # if the object is close to the placeholder
                self.current_inside.add(prim_path) # add to current set
                self.func_in(prim_path, key) # Do something while iterating

        self._whats_new_and_left() # Update new/left entry

        self.func_out() # Do something after iterating
        self._update_set()
        

class robot_proximity_checker(Proximity_Checker): # Check if object close to robot(Franka)
    def __init__(self, placeholder_prim_path, threshold):
        super().__init__(placeholder_prim_path, threshold)
        
        self.dp_cnt, self.p_cnt = 0, 0 # dirty/clean plates amount

        # Robot will despawn dirty plates
        self.dirty_plate_pt = [] # points
        self.dirty_plate_prim_paths = [] # prim_paths

    def summarize(self, log=False): # Count clean/dirty plate
        self.dp_cnt = 0
        self.p_cnt = 0

        for item in self.current_inside: # Iterate through close objects
            try:
                key, _ = pl.plates[item] # Check if the plate is clean/dirty (=key)
                if key == 'dp':
                    self.dp_cnt += 1
                elif key == 'p':
                    self.p_cnt += 1
                    
            except Exception as err:
                carb.log_warn(err)

        if log:
            carb.log_warn(f"REAL: DP: {self.dp_cnt}, P: {self.p_cnt}")

    def func_in(self, prim_path, key): # Do while iteration
        if key == 'dp': # if this plate is dirty
            self.dirty_plate_prim_paths.append(prim_path) # save prim path
            self.dirty_plate_pt.append(self.target_pt) # save point

    def func_out(self): # Do after iteration
        self.summarize(log=False)

    def func_prior(self): # Do before iteration
        # Reset
        self.dirty_plate_pt = []
        self.dirty_plate_prim_paths = []
        

class splitter_proximity_checker(Proximity_Checker): # For obtaining Image used on DL
    def __init__(self, placeholder_prim_path, threshold):
        super().__init__(placeholder_prim_path, threshold)

    def func_in(self, prim_path, key): # Do while iteration

        if len(self.entered) > 0: # If there's new entry
            carb.log_warn("Captured")
            decide_camera.capture() # Capture and save, create json file


<h2><font color='DimGray'>Camera</font></h2>
This shows and captures a scene from a certain veiw. Image file is generated with a json file.

In [ ]:
class cam:
    def __init__(self, camera_prim_path, vp, root_dir, show=True):
        self.camera_prim_path = camera_prim_path
        self.prim = stage.GetPrimAtPath(camera_prim_path)
        self.viewport = vp
        
        self.cnt = 0 # amount of captures made
        
        self.root_dir = root_dir # root
        self.image_path = None # currently saved image directory(path) in filesystem

        if not self.prim.IsValid():
            print(f"Camera prim {self.camera_prim_path} not found")
        else:
            print(f"Camera prim found: {self.camera_prim_path}")

        self.create_vp(show) # Setup viewport_window

        self.image = None
        
    def show(self, data: bool): # set view visible
        self.viewport_window.visible = data
    
    def create_vp(self, show): # Setup viewport_window
        self.viewport_window = vp_utils.create_viewport_window(self.viewport, usd_context.get_name(),640, 640)
        self.viewport_window.viewport_api.set_active_camera(self.camera_prim_path)
        self.show(show)
    
    def cheese(self): # Capture image and save
        self.image_path = f"{self.root_dir}/captured/{self.viewport}/frame_{self.cnt-2}.jpg"
        vp_utils.capture_viewport_to_file(self.viewport_window.viewport_api, self.image_path)

        self.cnt += 1 # increment

class decide_cam(cam): # Camera for Deep Learning Inference
    def __init__(self, camera_prim_path, vp, root_dir, show=True):
        super().__init__(camera_prim_path, vp, root_dir, show)
        
    def capture(self):
        self.cheese() # capture

        # save a json file, which the deep learning server will reference to choose its action
        # if processed == False, then deep learning server will make an inference
        json_struct = dict()
        json_struct['is_new'] = True
        json_struct['processed'] = False
        json_struct['file'] = self.image_path
        json_struct['dirty_plate'] = -1
        json_struct['clean_plate'] = -1

        try:
            with open(f"{self.root_dir}/json_folder/decide_camera.json", "w", encoding="utf-8") as f:
                json.dump(json_struct, f, ensure_ascii=False, indent=4)

        except Exception as err:
            carb.log_warn(err)

class robot_cam(cam): # Camera that show Frank Robot
    def __init__(self, camera_prim_path, vp, root_dir, show=True):
        super().__init__(camera_prim_path, vp, root_dir, show)

    def capture(self):
        self.cheese()

<h2><font color='DimGray'>Franka Robot</font></h2>
Franka Robot's End-Effector will move towards dirty plate and despawn it.

In [ ]:
from isaacsim.core.utils.numpy.rotations import euler_angles_to_quats
from omni.isaac.franka import Franka, KinematicsSolver

class franka_robot:
    def __init__(self):
        self.prim_path = "/World/robot_arm/Franka"
        
        self.franka = Franka(self.prim_path) # Get Franka Robot
        
        self.franka.set_solver_position_iteration_count(64)
        
        self.solver = KinematicsSolver(self.franka, None) # Kinematics Solver for Joints

        self.i = 0 # variable for smooth action (moves step-wise from initial position to final position)

    def startup(self): # Creates motion for preperation
        self.go_home(func_on_exit=lambda: self.get_ready(func_on_exit=lambda: self.test_position(func_on_exit= lambda: self.get_ready())))

    def go_home(self, step_size= 50, desired_time=1.0, func_on_exit=None): # Home Position
        self.move_to([0.3, -0.3, 0.35],[-180/180*np.pi,-90/180*np.pi,-90/180*np.pi], step_size=step_size, desired_time=desired_time, func_on_exit=func_on_exit)

    def get_ready(self, step_size= 50, desired_time=1.0, func_on_exit=None): # Ready Position
        self.move_to([0., 0.3, 0.8],[-180/180*np.pi,-90/180*np.pi,90/180*np.pi], step_size=step_size, desired_time=desired_time, func_on_exit=func_on_exit)

    def test_position(self, step_size=50, desired_time=1.0, func_on_exit=None): # Position for Debugging
        self.move_to([0., 0.6, 0.35],[-180/180*np.pi,-50/180*np.pi,90/180*np.pi], step_size=step_size, desired_time=desired_time, func_on_exit=func_on_exit)
    
    def move_to(self, xyz, rpy=None, desired_time = 1.0, step_size = 5, func_on_exit = None): # Set Robot Position(Joints)
        current_pos = self.franka.get_joints_state().positions # Obtain Current Joint Space
        goal_pos  = np.array(xyz) # Desired Position x,y,z [m]
        if rpy is None: # Desired Roll, Pitch, Yaw
            goal_quat = None
        else:
            goal_quat = euler_angles_to_quats(np.array(rpy)) # roll,pitch,yaw -> quat

        joints, compute_success = self.solver.compute_inverse_kinematics(np.array(xyz), goal_quat) # Use Inverse Kinematics to convert XYZ Space to Joint Space

        target = [i for i in joints.joint_positions] # 7 Joint Angles
        target.extend([0, 0]) # End Effecetor Joint Value

        if compute_success: # If possible
            self.steps = np.linspace(current_pos, target, num=step_size) # get steps for smooth transition, if not this then the robot changes instantly

            dt = desired_time / step_size
            
            self.i = 0
            
            self.tmp = tmr(self.increment, dt, 'tmp', num=step_size, func_on_exit=func_on_exit) # on every dt, move one step more
            self.tmp.begin()

    def increment(self):
        self.franka.set_joint_positions(self.steps[self.i])
        self.i += 1

<h2><font color='Purple'>Overall System</font></h2>
This is Finite State Machine, defining how conveyors move and how robot acts.

In [ ]:
class controller:
    def __init__(self):
        self.robot_moving = False
        self.current_plate_prim = None

        self.quarantine = False
        self.quarantine_tmr = tmr(lambda: None, 9.0, 'quarantine_tmr', num=1, func_on_exit = self.set_pass)

    def set_robot_moving(self):
        self.robot_moving = True
    def set_robot_stopped(self):
        self.robot_moving = False
    def gotcha(self): # Robot has arrived at the dirty plate
        carb.log_warn(self.current_plate_prim)
        pl.force_despawn(self.current_plate_prim) # despawn the dirty plate
        cv.run(['04'])
        fr.get_ready(step_size=50, func_on_exit=self.set_robot_stopped) # Move to ready position

    def decision_read(self): # Read json file
        with open(f"{root_dir}/json_folder/decide_camera.json", "r", encoding="utf-8") as f:
            data = json.load(f)

        is_new = data['is_new']
        processed = data['processed']
        fname = data['file']
        dp, p = data['dirty_plate'], data['clean_plate']

        if processed and is_new: # If Deep Learning Server has made an inference
            data['is_new'] = False
            try:
                with open(f"{root_dir}/json_folder/decide_camera.json", "w", encoding="utf-8") as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)
    
            except Exception as err:
                carb.log_warn(err)

            return True, [dp, p]

        else:
            return False, [-1, -1]

    def set_quarantine(self):
        self.quarantine = True
    def set_pass(self):
        self.quarantine = False
        cv.let_it_go()

    def should_switch(self): # Switch conveyorbelt direction according to inference
        new_inference, [dp, p] = self.decision_read() # Get Inference

        if new_inference: # If new inference has been made
            if dp > 0: # If there's dirty plate
                self.set_quarantine()
                cv.quarantine()

                if self.quarantine_tmr.running:
                    self.quarantine_tmr.reset()
                else:
                    self.quarantine_tmr.begin()

                carb.log_warn("Quarantine")
            elif p > 0:
                if not self.quarantine:
                    cv.let_it_go()
                    carb.log_warn("Let it go")

    def should_robot_pick(self): # If there's a dirty plate near robot, then robot moves towards it
        entries = len(robot_prox_checker.current_inside) # How many plates are near robot

        if entries > 0: # If there's at least one
            if robot_prox_checker.dp_cnt > 0: # If there's dirty plate
                cv.stop(['04']) # Stop Conveyorbelt
                robot_pos = physx_utils.get_world_position(stage, fr.prim_path) # Get Robot Position

                for prim, pt in zip(robot_prox_checker.dirty_plate_prim_paths, robot_prox_checker.dirty_plate_pt):    
                    if not self.robot_moving: # If robot is not operating
                        self.set_robot_moving()
                        self.current_plate_prim = prim # Target dirty plate
                        
                        desired_vec = pt - robot_pos # Get relative vector (World->Robot), There's no rotaion and only translation exists, so the rotational matrix is Identity
                        desired_vec = [desired_vec[0],desired_vec[1],0.35] # Z position is static

                        # Move end_effector
                        fr.move_to(desired_vec,[-180/180*np.pi,-50/180*np.pi,90/180*np.pi], step_size=50, desired_time = 1, func_on_exit=self.gotcha)
            else:
                cv.run(['04'])
                

<h1>Simulation Begin</h1>
Run Simulation

**1. Get required class objects**

In [ ]:
decide_camera = decide_cam("/World/Camera", "decide_camera", root_dir, True) # Decision Camera, show = True
robot_camera = robot_cam("/World/camera_region/Camera", "robot_camera", root_dir, True) # Robot Camera, show = False

Camera prim found: /World/Camera
Camera prim found: /World/camera_region/Camera

In [ ]:
cv = conveyors() # conveyor controller
pl = plates_spawn_kill() # plate (de)spawner
bt = bottle_spawn_kill(max_objs_at_once=50) # bottle (de)spawner

robot_prox_checker = robot_proximity_checker("/World/proximity_checker/for_robot",
                                               0.33) # robot proximity checker, threshold = 0.33m
splitter_prox_checker = splitter_proximity_checker("/World/proximity_checker/for_splitter",
                                                     0.2) # decision proximity checker, threshold = 0.2m

fr = franka_robot() # Franka Robot
ct = controller() # FSM

**2. Create Required Timers(Callbacks)**

In [ ]:
spawn_dirty_plate_tmr = tmr(pl.spawn_dirty_plate, 25.0, 'spawn_dirty_plate_tmr') # Spawns dirty plate every 10 seconds
spawn_clean_plate_tmr = tmr(pl.spawn_clean_plate, 10.0, 'spawn_clean_plate_tmr') # Spawns clean plate every 5 seconds
despawn_plates_tmr = tmr(pl.should_despawn, 2.0, 'despawn_plates_tmr') # Despawns plate that are on the ground every 2 seconds

bottle_spawner_tmr = tmr(bt.spawn_between_two, 1.0, 'bottle_spawner_tmr') # Spawns a bottle every 2.0 seconds
bottle_despawner_tmr = tmr(bt.should_despawn, 0.9, 'bottle_despawner_tmr') # Despawn exceeded amount of bottle every 1.9 seconds

robot_proximity_checker_tmr = tmr(robot_prox_checker.run, 0.2,
                                  "robot_proximity_checker_tmr") # Checks if plates are near robot every 0.2 seconds
robot_pick_tmr = tmr(ct.should_robot_pick, 0.2, 'robot_pick_tmr') # Robot will despawn dirty plates near it every 0.5 seconds 

splitter_proximity_checker_tmr = tmr(splitter_prox_checker.run, 0.1,
                                     "splitter_proximity_checker_tmr") # Checks if plates are under camera every 0.3 seconds
splitter_tmr = tmr(ct.should_switch, 0.1, 'splitter_tmr') # Changes conveyor direction according to DL inference every 0.3 seconds

**3. Intialize World and Timer**

In [ ]:
world.reset() # Reset Simulation

In [ ]:
fr.franka.initialize() # Initialize Franka
fr.startup() # Startup

In [ ]:
spawn_dirty_plate_tmr.begin()
spawn_clean_plate_tmr.begin()
despawn_plates_tmr.begin()

bottle_spawner_tmr.begin()
bottle_despawner_tmr.begin()

robot_proximity_checker_tmr.begin()
robot_pick_tmr.begin()

splitter_proximity_checker_tmr.begin()
splitter_tmr.begin()

**3.1 Summon as needed**

In [ ]:
pl.spawn_clean_plate("start")

In [ ]:
pl.spawn_dirty_plate("start")

**4. Stop Simulation and destroy**

In [ ]:
world.stop()
#world.pause()

In [ ]:
def run(func):
    try:
        func()
    except Exception as err:
        print(err)

run(pl.all_despawn)
run(bt.all_despawn)

run(spawn_dirty_plate_tmr.stop)
run(spawn_clean_plate_tmr.stop)
run(despawn_plates_tmr.stop)

run(bottle_spawner_tmr.stop)
run(bottle_despawner_tmr.stop)

run(robot_proximity_checker_tmr.stop)
run(robot_pick_tmr.stop)

run(splitter_proximity_checker_tmr.stop)
run(splitter_tmr.stop)

#del cv
#del pl
#del bt

#del robot_prox_checker
#del splitter_prox_checker

#del decide_camera
#del robot_camera

#del fr
#del ct

#del spawn_dirty_plate_tmr
#del spawn_clean_plate_tmr
#del despawn_plates_tmr

#del bottle_spawner_tmr
#del bottle_despawner_tmr

#del robot_proximity_checker_tmr
#del robot_pick_tmr

#del splitter_proximity_checker_tmr
#del splitter_tmr